In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
from math import sin
import pandas as pd
import os
import numpy as np

In [2]:
accountDB = pd.read_csv('data/account.csv', sep=";")
cardDB = pd.read_csv('data/card_train.csv', sep=";")
dispositionDB = pd.read_csv('data/disp.csv', sep=";")
districtDB = pd.read_csv('data/district.csv', sep=";")
loanDB = pd.read_csv('data/loan_train.csv', sep=";")
transactionDB = pd.read_csv('data/trans_train.csv', sep=";")

/home/iohan-sardinha/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
clientDB = pd.read_csv('data/client.csv', sep=";")
clientDB.insert(0, "women",(((clientDB['birth_number'] - ((clientDB['birth_number']//10000 * 10000) + (clientDB['birth_number'] - clientDB['birth_number']//100 * 100)))//100) > 50))
clientDB.insert(0,"age", 1998 - (1900 + clientDB['birth_number']//10000))
clientDB["birthday"] = np.where(clientDB["women"], clientDB['birth_number']-5000, clientDB['birth_number'])
clientDB = clientDB.drop(columns="birth_number")

In [4]:
loanDB = loanDB.rename(columns={"date":"loan_date", "duration":"loan_duration", "status": "loan_success"})
dispositionDB = dispositionDB.rename(columns={"type":"disposition_type"})
accountDB = accountDB.rename(columns={"date":"account_creation", "frequency":"account_frequency"})

In [47]:
clientWthDistrict = clientDB.merge(districtDB, left_on="district_id", right_on="code ")
clientWthDistrict = clientWthDistrict.drop(columns=["name ", "district_id"]) 
clientWthDistrict =  clientWthDistrict.rename(columns={col:("client_district "+col) for col in clientWthDistrict.columns[4:]})

In [46]:
accountWthDistrict = accountDB.merge(districtDB, left_on="district_id", right_on="code ")
accountWthDistrict = accountWthDistrict.drop(columns=["name ", "district_id"])
accountWthDistrict =  accountWthDistrict.rename(columns={col:("account_district "+col) for col in accountWthDistrict.columns[3:]})

In [89]:
accountClient = dispositionDB.merge(accountWthDistrict)
accountClient = accountClient.merge(clientWthDistrict)
accountClient = accountClient.drop(columns="client_id")
accountClient = accountClient[accountClient["disposition_type"] == "OWNER"]
accountClient.to_csv("accountClient.csv")

In [97]:
loanDB.to_csv("loan.csv")
loanFinal = loanDB.merge(accountClient, on="account_id")
loanFinal.to_csv("loanFinal.csv")
loanFinal

,loan_id,account_id,loan_date,amount,loan_duration,payments,loan_success,disp_id,disposition_type,account_frequency,...,client_district no. of municipalities with inhabitants 2000-9999,client_district no. of municipalities with inhabitants >10000,client_district no. of cities,client_district ratio of urban inhabitants,client_district average salary,client_district unemploymant rate '95,client_district unemploymant rate '96,client_district no. of enterpreneurs per 1000 inhabitants,client_district no. of commited crimes '95,client_district no. of commited crimes '96
0,5314,1787,930705,96396,12,8033,-1,2166,OWNER,weekly issuance,...,8,2,10,81.8,9650,3.38,3.67,100,2985,2804
1,5316,1801,930711,165960,36,4610,1,2181,OWNER,monthly issuance,...,7,3,10,73.5,8369,1.79,2.31,117,2854,2618
2,6863,9188,930728,127080,60,2118,1,11006,OWNER,monthly issuance,...,6,1,5,53.5,8390,2.28,2.89,132,2080,2122
3,5325,1843,930803,105804,36,2939,1,2235,OWNER,monthly issuance,...,10,1,9,74.8,10045,1.42,1.71,135,6604,6295
4,7240,11013,930906,274740,60,4579,1,13231,OWNER,weekly issuance,...,5,1,5,50.5,8288,3.79,4.52,110,1562,1460
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
323,6818,9030,961212,155616,48,3242,1,10813,OWNER,monthly issuance,...,7,4,4,62.6,8994,3.80,4.79,110,9672,9208
324,5625,3189,961215,222180,60,3703,-1,3855,OWNER,monthly issuance,...,5,1,6,55.6,8843,2.82,3.60,113,818,888
325,6805,8972,961221,45024,48,938,1,10742,OWNER,monthly issuance,...,8,5,7,89.9,10177,6.63,7.75,81,9878,10108
326,7233,10963,961225,115812,36,3217,1,13172,OWNER,monthly issuance,...,10,1,8,56.9,8427,1.12,1.54,107,1874,1913
